# Julia and the Power of Multiple Dispatch

Julia is built around the concept of *multiple dispatch*. Multiple dispatch means that when you call a function 

```juila
f(a, b, c)
```

Julia chooses *which implementation* of the function `f` to call based on the types of `a`, `b`, and `c`. Compare this with Python, in which

```python
# python
f(a, b, c)
```

always calls the same function `f` for any `a`, `b`, and `c`,

or

```python
# python
a.f(b, c)
```

which chooses the version of `f` based on the class of `a`, but not `b` or `c`. 

This allows us to do some really cool things in Julia that would be difficult in other languages. As an example, let's show off by building a simple tool for automatic differentiation. 

By the way, if you want to use automatic differentiation in your own code, the excellent [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) package provides a complete implementation of the basic idea that we're showing off here. 

# Example: Automatic Differentiation

A common problem in computing (especially in robotics) is:

Given a function $f(x)$, find the function $df(x)$ which evaluates $\frac{d f}{d x}$ at x. 

When $f(x)$ is a mathematical expression, this is straightforward: we just work out the derivative analytically and write it down. But what do we do when $f(x)$ is a function defined in code? 

We have a few options: 

### Analytic differentiation

If we have access to the code of $f(x)$ or the math it was based on, we can work out its derivative by hand and then write code to implement that derivative. This works fine, but it's wasteful (we end up writing code to implement both $f$ and $df$) and error-prone (we have to make sure both pieces of code match...forever). 

### Numerical differentiation

If we don't want to write out $df$ by hand, we can estimate the derivative at some input $x$ by perturbing that input slightly: 

$$
\left. \frac{d f}{dx} \right|_x \approx \frac{f(x + \delta x) - f(x)}{\delta x}
$$

Ideally, this gives us the exact answer as $\delta x \to 0$. Unfortunately, computers are not mathematically ideal. In particular, as $\delta x$ shrinks, the error induced by floating-point rounding becomes more significant. On the other hand, as $\delta x$ grows, our estimate of the derivative becomes less mathematically ideal. 

### Automatic differentiation

Automatic differentiation (or autodiff) refers to a family of methods for computing exact derivatives, generally by running a special data type through your code. 

In forward-mode autodiff (the only kind we'll discuss here), we create a *dual* number, which stores both its value and its derivative. Then we implement basic mathematical operations on that dual number and let the chain rule take care of the rest. 

This is easier to see with an example. 

First, we'll create a structure to represent our dual numbers: 

In [1]:
# The {T} annotation indicates that our `Dual` type is 
# parameterized (i.e. generic). This struct can hold 
# any pair of value and derivative, as long as both are
# of the same type. 

struct Dual{T} <: Number
    value::T
    derivative::T
end

# For example, we can construct a `Dual` with Float64 entries:
Dual(1.0, 0.0)

Dual{Float64}(1.0, 0.0)

In [2]:
# or a Dual with integer entries:
Dual(1, 0)

Dual{Int64}(1, 0)

In [3]:
# and Julia will generate specialized code for every new kind of `Dual{T}` we use. 

Now let's implement addition. Julia already knows how to add numbers, we just have to teach it to also add Duals. First, we need to import the `+` function so that we can extend its behavior:

In [4]:
import Base: +  # Base is the common library of tools and functions that ship with Julia

Functions in Julia can have lots of different methods that act on different input types. To see all the existing ways you can use a given function, use `methods()`: 

In [5]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:355
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:353
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:391
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:28
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:80
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) 

We're just going to add a few new methods for `+` to deal with `Dual`s: 

In [6]:
+(n::Number, d::Dual)   = Dual(d.value + n,         d.derivative)
+(d::Dual,   n::Number) = Dual(d.value + n,         d.derivative)
+(d1::Dual,  d2::Dual)  = Dual(d1.value + d2.value, d1.derivative + d2.derivative)

+ (generic function with 166 methods)

Note that `Number` is an abstract type, representing an entire family of types. By defining `+` for `Dual` and `Number`, we get the right behavior for any subtype of `Number`, like `Int`, `Float64`, `UInt8`, `BigInt`, etc.

Now we can add Duals, and they will propagate their derivative information: 

In [7]:
d1 = Dual(1.0, 1.0)

d1 + 5

Dual{Float64}(6.0, 1.0)

In [8]:
d2 = Dual(1.0, 0.5)
d1 + d2

Dual{Float64}(2.0, 1.5)

Let's implement multiplication using the product rule from basic calculus: 

In [9]:
import Base: *
*(n::Number, d::Dual)   = Dual(n * d.value,         d.derivative * n)
*(d::Dual,   n::Number) = Dual(n * d.value,         d.derivative * n)
*(d1::Dual,  d2::Dual)  = Dual(d1.value * d2.value, d1.value * d2.derivative + d2.value * d1.derivative)

* (generic function with 346 methods)

### Aside: Why Julia is Different

We can do this kind of operator overloading in many languages. But the cool part about Julia is that there's nothing special about `+` or other operators. We can overload any function we want for any combination of types. Let's implement a few more functions for our `Dual` type:

In [10]:
import Base: sin, cos

sin(d::Dual) = Dual(sin(d.value),  cos(d.value) * d.derivative)
cos(d::Dual) = Dual(cos(d.value), -sin(d.value) * d.derivative)

cos (generic function with 13 methods)

### Back to Autodiff

Now we can use this `Dual` number to perform automatic differentiation! First, we need a function to differentiate:

In [11]:
f(x) = sin(3x + 5)

f (generic function with 1 method)

To evaluate the derivative of `f` at `x`, we first create a Dual with value `x` and derivative $\frac{d x}{d x} = 1$

In [12]:
x = π / 4
x_dual = Dual(x, 1.0)

Dual{Float64}(0.7853981633974483, 1.0)

Now we just pass our `Dual` through the function and check the `derivative` field of the result:

In [13]:
y = f(x_dual)
df = y.derivative
println("The derivative of f at $x is $df")

The derivative of f at 0.7853981633974483 is 1.4324472070543601


We can compare that against numerical differentiation: 

In [14]:
δx = 1e-8
estimate = (f(x + δx) - f(x)) / (δx)
println("The derivative of f at $x is approximately $estimate")

The derivative of f at 0.7853981633974483 is approximately 1.4324471608873068


Success! We got an exact measurement (up to machine accuracy) of the derivative of `f` without any numerical differentation. The best part is that in Julia, this kind of work is extremely efficient: 

In [15]:
# Tell Julia to use the packages specified in the Project.toml
# and Manifest.toml files contained in the DynamicWalking2018.jl
# repository. This ensures that you will have the same packages
# that were used to create this tutorial. 
using Pkg
pkg"activate .."
pkg"instantiate"

# Load BenchmarkTools.jl, which provides the @btime macro 
# for benchmarking code in Julia
using BenchmarkTools: @btime

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `git@github.com:JuliaRegistries/General.git`


In [16]:
@btime f($x)

  8.619 ns (0 allocations: 0 bytes)


0.87864131216594

In [17]:
@btime f($x_dual)

  17.259 ns (0 allocations: 0 bytes)


Dual{Float64}(0.87864131216594, 1.4324472070543601)

There's no overhead introduced by our Dual type: computing `f` *and its derivative* takes only twice as long as computing the value of `f`. 

## More operations

Multiple dispatch is the basis for much of Julia's power and flexibility. We can define new interesting types, give them behaviors, and then use those behaviors to do more complex things. As a very basic demonstration, now that we've defined multiplication and addition for our Dual type, we get sums and exponentiation for free:

In [18]:
d = Dual(0.5, 1.0)
(d + 1)^2

Dual{Float64}(2.25, 3.0)

In [19]:
sum([d, 2 * d, d^2])

Dual{Float64}(1.75, 4.0)

# Completing the Implementation

To make a real autodiff library, we just need to automate the process of creating a new Dual, passing it through a function, and extracting its derivative. Functions in Julia are first-class, so it's easy to create a function that operates on other functions:

In [20]:
function derivative(f, x)
    x_dual = Dual(x, one(x))  # one(x) gives the unit value of whatever x's type is. So, for an integer x, one(x) == 1
    y_dual = f(x_dual)
    y_dual.derivative
end

derivative (generic function with 1 method)

In [21]:
derivative(f, π/4)

1.4324472070543601

In [22]:
f2(x) = x^2
derivative(f2, 2.0)

4.0

In [23]:
# Check against the analytical derivative
@assert derivative(f2, 2.0) ≈ 2 * 2.0

In [24]:
f3(x) = 3x + sin(x^2 * 10) + cos(x)
derivative(f3, 0.5)

-5.49086169407354

In [25]:
# Check against the analytical derivative
@assert derivative(f3, 0.5) ≈ 3 + cos(0.5^2 * 10) * (10 * 2 * 0.5) - sin(0.5)

To get even more fancy, we can make `derivative(f)` return a *new function* `df` which evaluates the derivative of `f`:

In [26]:
derivative(f) = x -> derivative(f, x)  # the -> syntax creates an anonymous function

derivative (generic function with 2 methods)

In [27]:
df = derivative(f)

#5 (generic function with 1 method)

In [28]:
df(π/4)

1.4324472070543601

And despite the use of functions-as-data and anonymous functions, we still haven't lost any performance! Using `df()` is just as fast as our original `f(x_dual)`: 

In [29]:
@btime $df($x)

  16.600 ns (0 allocations: 0 bytes)


1.4324472070543601

## Conclusion

With relatively few method implementations, multiple dispatch lets us build an entire library for differentiating complex functions efficiently. 